## 1.贪心

### 1.1 会议安排
一个会议的开始结束时间[x,y]，怎样安排会议室让会议最多

In [1]:
#思路 选结束时间最早的

In [2]:
class Program:
    def __init__(self,s,e):
        self.start = s
        self.end = e

In [4]:
def bestArrange(programs):
    timeline = 0
    result = 0
    arr = sorted(programs,key=lambda x:x.end)
    for i in arr:
        if timeline <= i.start:
            result+=1
            timeline = i.end
    return result

### 1.2 切金条
一块金条由长度为N的正数数组，长度为数组和sum。要求将金条切成最终数组形式的N段，每次只能切一刀，成本是被切的长度。求成本最小的切法下成本是多少。例如 [10,30,20]， 60长度的金条可以切30、30，然后再将30切成10、20，总成本是90。（另一种切法是60=>10、50；50=>20、30，成本是110）。

In [29]:
'''
思路：哈夫曼编码
1.准备小根堆，将数组的数依次压入。
2.每次弹出两个数，将两个数合成，再压回小根堆。
3.直到小根堆只剩一个数停止。 每次第二步合成的值，即为成本
'''
import heapq
def lessMoney(arr):
    pq = list()
    for i in arr:
        heapq.heappush(pq, i)
    res = 0
    cur = 0
    while len(pq) > 1:
        cur = heapq.heappop(pq) + heapq.heappop(pq)
        res += cur
        heapq.heappush(pq, cur)
    return res
        

In [30]:
lessMoney([10,20,30])

90

### 1.3 最大项目收益
* 输入：正数数组costs、profits、正数K、M
* cost[i]、profits[i]表示i号项目的花费与收益(扣除过花费)，K表示最多只能串行K个项目。M表示初始资金。
* 每做完一个项目，马上获得收益，可以支持下一个项目中。不能并行做项目。
* 输出最后获得的最大钱数

In [38]:
'''
思路：
把所有项目压入小根堆按花费排序，手里的钱满足花费的所有项目弹出，压入按收益排序的大根堆。
每个时间节点，大根堆内不为空，弹出一个项目，加上收益。然后再看小根堆是否有新满足的项目弹出、压入大根堆。
直到大根堆内没有项目或已完成K个项目，停止
'''
class Cost:
    def __init__(self, index, cost):
        self.index = index
        self.cost = cost
    def __lt__(self, other):
        return self.index < other.index if self.cost == other.cost else self.cost < other.cost

class Profit:
    def __init__(self, index, profit):
        self.index = index
        self.profit = profit
    def __lt__(self, other):
        return self.index < other.index if self.profit == other.profit else self.profit > other.profit

def mostMoney(costs, profits, K, M):
    allPro = list()
    waitPro = list()
    earn = M
    for i in range(len(costs)):
        heapq.heappush(allPro, Cost(i, costs[i]))
    while allPro and heapq.nsmallest(1, allPro)[0].cost <= M:
        cur = heapq.heappop(allPro)
        heapq.heappush(waitPro, Profit(cur.index, profits[cur.index]))
    for time in range(K):
        if waitPro:
            cur = heapq.heappop(waitPro)
            earn += cur.profit
            if allPro:
                while allPro and heapq.nsmallest(1, allPro)[0].cost <= earn:
                    cur = heapq.heappop(allPro)
                    heapq.heappush(waitPro, Profit(cur.index, profits[cur.index]))
        else:
            return earn
    return earn


In [40]:
costs =   [3,6,14,8,9,10]
profits = [3,2,5,1,2,1]
K = 4
M = 3
mostMoney(costs, profits, K, M)

11

### 1.4 点灯
给定一个字符串，只由"X"和"."两种字符构成。
* "X"表示墙， 不能放灯，也不需要被点亮,
* "."表示居民点，可以放灯，需要被点亮。
* 如果灯放在i位置，可以让i-1、i和i+1三个位置被点亮。
* 返回如果想点亮str中所有需要点亮的位置，至少需要几盏灯。

In [42]:
'''
思路：
如果i位置是X，跳过；
如果i位置是. i+1位置是X            一定i放灯， 跳到i+2
           i+1位置是. i+2位置是x，    i放灯，跳到i+3 
           i+1位置是. i+2位置是.， 一定i+1放灯，跳到i+4
'''
def minLight(road):
    roadList = list(road)
    index = 0
    light =0
    while index < len(roadList):
        if roadList[index] == "X":
            index += 1
        else:
            light += 1
            if index + 1 == len(roadList):
                break
            else:
                if roadList[index + 1] == "X":
                    index = index + 2
                else:
                    index = index + 3
    return light

## 2. 并查集
1. isSameSet(A,B): AB是否属于同一集合
2. union(A,B): 将AB聚合为一个集合
3. 单次查询平均O(1)

In [50]:
# size小挂大,保证链长度增长最慢慢
# union时小的代表节点 指向大的代表节点
# findAncestor时，不光返回代表节点，也把沿途遇到的父节点直接指向查到代表节点，保证下次查询不会再遍历（加速，只痛一次

In [43]:
class Node:
    def __init__(self, val):
        self.val = val

In [49]:
from collections import deque
class UnionSet:
    def __init__(self, values):
        self.nodes = dict()
        self.parents = dict() #(node:parentNode)
        self.sizeMap = dict()    #(representativeNode:size)
        for cur in values:
            node = Node(cur)
            self.nodes[cur] = node
            self.parents[node] = [node]
            self.sizeMap[node] = 1
    #给一个节点，找到代表节点返回（每次查找后，把沿途node的父亲指向代表节点
    def findAncestor(node):
        path = deque()
        cur = node
        while cur != self.parents.get(cur):
            path.append(cur)
            cur = self.parents.get(cur)
        while len(path) > 0:
            self.parents[path.pop()] = cur
        return cur
    
    def isSameSet(a, b): #传入为value而非node
        return findAncestor(self.nodes.get(a)) == findAncestor(self.nodes.get(b))
    
    def union(a, b): #传入为value而非node
        aHead = findAncestor(self.nodes.get(a))
        bHead = findAncestor(self.nodes.get(b))
        if aHead != bHead:
            aSetSize= self.sizeMap.get(aHead)
            bSetSize= self.sizeMap.get(bHead)
            if aSetSize >= bSetSize:
                big = aHead
                small = bHead
            else:
                big = bHead
                small = aHead
            self.parents[small] = big
            self.sizeMap[big] = aSetSize + bSetSize
            self.sizeMap.pop(small)
            
        

In [48]:
a = dict({1:3,2:4})
a.pop(1)

3